# Pre-processing

## Import nécessaire

In [58]:
# import them
import pandas as pd
import numpy as np
import os
import math

## Construction du dataset

#### Chargement des données

In [59]:
#load the data
data_folder = "./data/"

list_of_files = [pd.read_excel(f'{data_folder}{f}') for f in os.listdir(data_folder) if f.endswith('.xlsx')]
for i in range(len(list_of_files[0].columns)):
    i_column = []
    for df in list_of_files:
        i_column.append(df.columns[i])
    #print all unique values in the list
    if len(set(i_column)) > 1:
        print(f"{i_column} has multiple values")
        print("\n")

In [60]:
# merge the dfs
df = pd.concat(list_of_files, axis=0, ignore_index=True)

#### Elimination des colonnes inutiles

## Renommage des entités

* colonnes explicite (Ry -> rayon)
* label explicites (IR -> infra-rouge)

In [61]:
# rename a column
df.rename(columns={"# of Gears": "Number of Gears"}, inplace=True)
df.rename(columns={"# of Cylinders and Rotors": "Number of Cylinders and Rotors"}, inplace=True)
df.rename(columns={"Test Veh Configuration #": "Test Veh Configuration"}, inplace=True)
df.rename(columns={"Transmission Overdrive Desc": "Gear Ratio"}, inplace=True)
df.rename(columns={"Tested Transmission Type": "Transmission Type"}, inplace=True)



In [62]:
#on repère les entités en str et on explicite leurs labels
for column in df.columns:
    if type(df.loc[0, column]) == str:
        unique_values = df[column].unique()
        print(f"{column} has {len(unique_values)} unique values:\n {unique_values[:10]}...\n")

Vehicle Manufacturer Name has 51 unique values:
 ['aston martin' 'Audi' 'Bentley' 'BMW' 'BYD Motors Inc.' 'CSC' 'FCA Italy'
 'FCA US LLC' 'Ferrari' 'FOMOCO']...

Veh Mfr Code has 50 unique values:
 ['ASX' 'ADX' 'BEX' 'BMX' 'BYD' 'CSC' 'FTG' 'CRX' 'FEX' 'FMX']...

Represented Test Veh Make has 89 unique values:
 ['Aston Martin' 'AUDI' 'BENTLEY' 'BMW' 'Mini' 'BYD' 'BUICK' 'CADILLAC'
 'Fiat' 'Alfa Romeo']...

Represented Test Veh Model has 2163 unique values:
 ['DB11' 'Rapide S' 'V12 Vantage S' 'Vanquish' 'A4' 'Q5' 'RS7' 'S5'
 'Continental GT' '230i Convertible']...

Test Vehicle ID has 3984 unique values:
 ['500TT5148' '410TT4202' '288TT2087' '298VP2156' '314TT3175' 'D3UB-BMQ'
 'D3UF-Q5A' 'DFUB-Q5A' 'E3UJ-RSQ' 'D3UF-BAQ']...

Actual Tested Testgroup has 2218 unique values:
 ['HASXV05.2AM5' 'FASXV06.0VHB' 'HASXV06.0VHB' 'DADXV02.03UB'
 'DADXJ03.03UF' 'DADXT02.04UB' 'DADXJ02.0FUB' 'EADXV04.03UJ'
 'DBEXV06.04UC' 'HBMXV02.0B4X']...

Vehicle Type has 3 unique values:
 ['Car' 'Both' 'Truck']..

## Passage en unité plus lisibles

* unité internationales
* ordinnal -> nombre
* nominal -> nombre (Hot Encoding)
* modification des types (string -> int ...)


### unité internationale

In [63]:
unit_convert = {
    "miles to kilometer": 1.60934,
    "lbs to kg": 0.453592,
}

df["litre pour 100km"] = df.apply(lambda x: 235.214583/x["RND_ADJ_FE"] if x["FE_UNIT"] == "MPG" and x["RND_ADJ_FE"]>0 else 0, axis=1)
df["Weight (kg)"] = df["Equivalent Test Weight (lbs.)"]*unit_convert["lbs to kg"]

In [ ]:
# spécifiquement à notre dataset
# doing our own research we cam out with:
CO2_coef = {
    "Carbon Dioxide	": 1,
    "CO2": 1,
    "Carbon Monoxide": 1,
    "CO": 1,
    "Nitrogen Oxides": 298,
    "NOx": 298,
    "N2O": 298,
    
    "Methane": 25,  
    "CH4": 25,
    "Hydrocarbons": 1,
    "THC": 1,
}

cols = ["THC (g/mi)", "CO (g/mi)", "CO2 (g/mi)", "NOx (g/mi)", "CH4 (g/mi)", "N2O (g/mi)", "PM (g/mi)"]
for col in cols:
    df[col] = df[col].apply(lambda x: 0 if math.isnan(x) else x)
    

df["CO2 (eqg/mi)"] = df.apply(lambda x:(
                              x["THC (g/mi)"]*CO2_coef["THC"]+
                              x["CO (g/mi)"]*CO2_coef["CO"]+
                              x["CO2 (g/mi)"]*CO2_coef["CO2"]+
                              x["NOx (g/mi)"]*CO2_coef["NOx"]+
                              x["CH4 (g/mi)"]*CO2_coef["CH4"]+
                              x["N2O (g/mi)"]*CO2_coef["N2O"])
                              , axis=1)

df["grammes equivalent CO2/km"] = df["CO2 (eqg/mi)"]*unit_convert["miles to kilometer"]

# dans notre cas on se permet d'enlever les colonnes
df.drop(columns=["THC (g/mi)", "CO (g/mi)", "CO2 (g/mi)", "NOx (g/mi)", "CH4 (g/mi)", "N2O (g/mi)", "PM (g/mi)", "CO2 (eqg/mi)"], inplace=True)

### Ordinal

In [65]:
df["Fuel type"] = df["Test Fuel Type Description"].apply(lambda x: 
    "Electricity" if "Electricity" in x else
    "Hydrogen" if "Hydrogen" in x else 
    "E85" if "E85" in x else 
    "Diesel Cold" if "Cold" in x and "Diesel" in x else 
    "Diesel" if "Diesel" in x else 
    "E10 Premium Cold" if "Cold" in x and "E10" in x and "Premium" else
    "E10 Regular Cold" if "Cold" in x and "E10" in x and "Regular" else
    "E10 Premium" if "E10 Premium" in x or "" in x else
    "E10 Regular")
# on base cet ordinalité sur le densité énergétique et le rendement moyen (TODO: ajouter le rendement dans l'ordinalité)
fuel_cat_to_num = {"Hydrogen":1, "E85":2, "E10 Premium Cold":3, "E10 Premium":4, "Diesel Cold":5, "Diesel":6, "Electricity":7}
df["Fuel type"] = df["Fuel type"].apply(lambda x: fuel_cat_to_num[x])


df["Rear Type"] = df["Drive System Description"].apply(lambda x:
    "Both" if "Partime" in x else
    "front" if "Front" in x else
    "back" if "Rear" in x else
    "4WD")
rear_type_cat_to_num = {"Both": 0.5, "front": 0, "back": 0.1, "4WD": 1}
df["Rear Type"] = df["Rear Type"].apply(lambda x: rear_type_cat_to_num[x])


df["Number of wheel"] = df["Drive System Description"].apply(lambda x:
    2 if "2" in x else
    4 if "4" in x else
    0.5 if "Part" in x else
    4)

df["Equivalent Test Weight (kg)"] = df["Equivalent Test Weight (lbs.)"].replace(',', '', regex=True)



gear_ratio_cat_to_num = {"Top gear ratio < 1": 0, "No gear ratio < 1": 1}
df["Gear ratio"] = df["Gear Ratio"].apply(lambda x: gear_ratio_cat_to_num[x])


df.drop(columns=["Equivalent Test Weight (lbs.)", "RND_ADJ_FE", "FE_UNIT", "Test Fuel Type Description", "Drive System Description"], inplace=True)


### Nominal

In [66]:
#nominal à numéro
from sklearn.preprocessing import OneHotEncoder

def mean_encoding(df, col):
    mean_encoding = df.groupby(col)['litre pour 100km'].mean()
    df[f"{col} Mean Encoding"] = df[col].map(mean_encoding)
    return df   

def onehotencoding(df, col):
    encoder = OneHotEncoder(sparse_output=False)
    encoded_features = encoder.fit_transform(df[[col]])
    encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out([col]))
    # Fusion des données encodées avec le DataFrame original
    df_encoded = pd.concat([df.drop(columns=[col]), encoded_df], axis=1)
    return df_encoded

df = mean_encoding(df, "Veh Mfr Code")
df = mean_encoding(df, "Transmission Type")
df = onehotencoding(df, "Rear Type")

### Type de variables

In [67]:
df["grammes equivalent CO2/km"] = df["grammes equivalent CO2/km"].astype(float)
df["Weight (kg)"] = df["Weight (kg)"].astype(float)

## Save result

In [68]:
# keep only numerical columns
df = df.select_dtypes(include=[np.number])
df.columns

Index(['Model Year', 'Test Veh Configuration', 'Test Veh Displacement (L)',
       'Rated Horsepower', 'Number of Cylinders and Rotors', 'Number of Gears',
       'Transmission Overdrive Code', 'Axle Ratio', 'N/V Ratio',
       'Shift Indicator Light Use Cd', 'ADFE Total Road Load HP',
       'ADFE Equiv. Test Weight (lbs.)', 'ADFE N/V Ratio', 'Test Procedure Cd',
       'Test Fuel Type Cd', 'FE Bag 1', 'FE Bag 2', 'FE Bag 3', 'FE Bag 4',
       'DT-Inertia Work Ratio Rating', 'DT-Absolute Speed Change Ratg',
       'DT-Energy Economy Rating', 'Target Coef A (lbf)',
       'Target Coef B (lbf/mph)', 'Target Coef C (lbf/mph**2)',
       'Set Coef A (lbf)', 'Set Coef B (lbf/mph)', 'Set Coef C (lbf/mph**2)',
       'Averaging Weighting Factor', 'litre pour 100km', 'Weight (kg)',
       'CO2 (eqg/mi)', 'grammes equivalent CO2/km', 'Fuel type',
       'Number of wheel', 'Equivalent Test Weight (kg)', 'Gear ratio',
       'Veh Mfr Code Mean Encoding', 'Transmission Type Mean Encoding',
     

In [70]:
df.to_csv("./results/df_builded.csv", index=False)

a l'issu de ce jupyter, le jeu de donnée est:
* composé uniquement de nombres (int, float)
* complet
* lisible